# Byonoy Absorbance 96 Automate


<table style="width:100%; border-collapse:collapse;">
<tr>
<td style="width:60%; font-size:15px; line-height:1.7; vertical-align:top; padding-right:15px;">

<ul style="margin-top:0;">
  <li><a href="https://byonoy.com/absorbance-automate/" target="_blank"><b>OEM Link</b></a></li>
  <li><b>Communication Protocol / Hardware:</b> HID (USB-A/C)</li>
  <li><b>Communication Level:</b> Firmware</li>
  <li>VID:PID <code>16d0:1199</code></li>
  <li>Takes a single SLAS-format 96-wellplate on the base/detection unit, enables movement of cap/illumination unit over it, and reads all 96 wells simultaneously.</li>
</ul>

</td>

<td style="width:40%; text-align:center; vertical-align:middle;">
  <img src="img/byonoy_absorbance_96_automate.png" width="500"/><br>
  <i>Figure: Byonoy Absorbance 96 Automate - Illumination unit being moved onto detection unit</i>
</td>
</tr>
</table>

---
## Setup Instructions (Physical)

The Byonoy Absorbance 96 Automate is a an absorbance plate reader consisting of...
1. a `base` containing the liqht source,
2. a `reader_cap` containing the light detectors, and
3. a `cap_adapter` representing a simple resource_holder for the `reader_cap`

It requires only one cable connections to be operational:
1. USB cable (USB-C at `base` end; USB-A at control PC end)

---
## Setup Instructions (Programmatic)

If used with a liquid handler, first setup the liquid handler:

In [1]:
import logging
from pylabrobot.io import LOG_LEVEL_IO
from datetime import datetime

current_date = datetime.today().strftime('%Y-%m-%d')
protocol_mode = "execution"

# Create the shared file handler once
fh = logging.FileHandler(f"{current_date}_testing_{protocol_mode}.log", mode="a")
fh.setLevel(LOG_LEVEL_IO)
formatter = logging.Formatter(
    "%(asctime)s [%(levelname)s] %(name)s - %(message)s"
)
fh.setFormatter(formatter)

# Configure the main pylabrobot logger
logger_plr = logging.getLogger("pylabrobot")
logger_plr.setLevel(LOG_LEVEL_IO)
if not any(isinstance(h, logging.FileHandler) and h.baseFilename == fh.baseFilename
           for h in logger_plr.handlers):
    logger_plr.addHandler(fh)

# Other loggers can reuse the same file handler
logger_manager = logging.getLogger("manager")
logger_device = logging.getLogger("device")

for logger in [logger_manager, logger_device]:
    logger.setLevel(logging.DEBUG)  # or logging.INFO
    if not any(isinstance(h, logging.FileHandler) and h.baseFilename == fh.baseFilename
               for h in logger.handlers):
        logger.addHandler(fh)

# START LOGGING
logger_manager.info("START AUTOMATED PROTOCOL")


In [2]:
from pylabrobot.liquid_handling import LiquidHandler, LiquidHandlerChatterboxBackend
from pylabrobot.resources import STARDeck

lh = LiquidHandler(deck=STARDeck(), backend=LiquidHandlerChatterboxBackend())

In [3]:
await lh.setup()

Setting up the liquid handler.


Then generate a plate definition for the plate you want to read:

In [4]:
from pylabrobot.resources.coordinate import Coordinate
from pylabrobot.resources.cellvis.plates import CellVis_96_wellplate_350uL_Fb


plate = CellVis_96_wellplate_350uL_Fb(name='plate')
lh.deck.assign_child_resource(plate, location=Coordinate(0, 0, 0))

Now instantiate the Byonoy absorbance plate reader:

In [5]:
from pylabrobot.plate_reading.byonoy import (
    byonoy_absorbance_adapter,
    byonoy_absorbance96_base_and_reader
)

cap_adapter = byonoy_absorbance_adapter(name='cap_adapter')

base, reader_cap = byonoy_absorbance96_base_and_reader(name='base', assign=True)

lh.deck.assign_child_resource(cap_adapter, location=Coordinate(400, 0, 0))

In [6]:
await reader_cap.setup(verbose=True)

reader_cap.setup_finished

Connected to Bynoy Absorbance 96 Automate (via HID with VID=5840:PID=4505) on b'DevSrvsID:4308410804'
Identified available wavelengths: [420, 600] nm


True

In [7]:
reader_cap.backend.io.device_info

{'path': b'DevSrvsID:4308410804',
 'vendor_id': 5840,
 'product_id': 4505,
 'serial_number': 'BYOMAA00058',
 'release_number': 512,
 'manufacturer_string': 'Byonoy GmbH',
 'product_string': 'Absorbance 96 Automate',
 'usage_page': 65280,
 'usage': 1,
 'interface_number': 0,
 'bus_type': <BusType.USB: 1>}

In [8]:
reader_cap.backend.available_wavelengths

[420, 600]

## Test Movement for Plate Reading

In [9]:
cap_adapter, base, reader_cap

(ResourceHolder(name='cap_adapter', location=Coordinate(400.000, 000.000, 000.000), size_x=127.76, size_y=85.59, size_z=14.07, category=resource_holder),
 ByonoyBase(name='base_base', location=None, size_x=138, size_y=95.7, size_z=27.7, category=None),
 PlateReader(name='base_reader', location=Coordinate(000.000, 000.000, 010.660), size_x=138, size_y=95.7, size_z=0, category=None))

---

## Usage / Machine Features

### Query Machine Configuration

In [10]:
await reader_cap.backend.get_available_absorbance_wavelengths()

[420, 600]

### Measure Absorbance

In [11]:
readings_420_nested_list = await reader_cap.backend.read_absorbance(
    wells=plate.children[:55],
    wavelength = 420, # units: nm
    output_nested_list=True
)

import pandas as pd

pd.DataFrame(readings_420_nested_list)

,0,1,2,3,4,5,6,7,8,9,10,11
0,0.000002,-0.000002,0.000083,0.000038,0.000048,2.975314e-05,0.000075,None,None,None,None,None
1,0.000062,0.000051,0.000040,0.000018,0.000064,3.082320e-05,0.000044,None,None,None,None,None
2,0.000088,0.000055,0.000069,0.000009,0.000079,7.937726e-05,0.000078,None,None,None,None,None
3,0.000080,0.000050,0.000009,0.000069,0.000067,3.182423e-05,0.000070,None,None,None,None,None
4,0.000042,0.000003,0.000110,0.000005,-0.000005,-1.815412e-05,0.000070,None,None,None,None,None
5,0.000055,0.000054,-0.000023,0.000041,0.000036,9.664112e-07,0.000039,None,None,None,None,None
6,0.000046,0.000025,0.000019,0.000017,0.000039,3.658781e-05,0.000066,None,None,None,None,None
7,0.000038,0.000018,0.000055,0.000041,0.000034,-3.216584e-05,NaN,None,None,None,None,None


In [12]:
import time

In [13]:
start_time = time.time()

readings_600_nested_list = await reader_cap.backend.read_absorbance(
    wells=plate.children[:],
    wavelength = 600, # units: nm
    output_nested_list=True
)
display(pd.DataFrame(readings_600_nested_list))


time.time() - start_time

,0,1,2,3,4,5,6,7,8,9,10,11
0,0.000097,0.000079,0.000087,0.000092,0.000085,0.000097,0.000086,0.000088,0.000074,0.000111,0.000066,0.000076
1,0.000050,0.000074,0.000063,0.000054,0.000073,0.000066,0.000050,0.000061,0.000082,0.000095,0.000051,0.000059
2,0.000093,0.000049,0.000031,0.000081,0.000067,0.000083,0.000066,0.000104,0.000074,0.000064,0.000040,0.000069
3,0.000096,0.000074,0.000023,0.000075,0.000100,0.000053,0.000064,0.000087,0.000070,0.000073,0.000050,0.000054
4,0.000087,0.000074,0.000161,0.000070,0.000080,0.000069,0.000101,0.000106,0.000112,0.000103,0.000059,0.000062
5,0.000058,0.000067,0.000023,0.000068,0.000036,0.000053,0.000035,0.000044,0.000045,0.000097,0.000039,0.000033
6,0.000080,0.000036,0.000012,0.000079,0.000062,0.000061,0.000046,0.000084,0.000043,0.000050,0.000026,0.000064
7,0.000087,0.000053,0.000072,0.000060,0.000076,0.000031,0.000034,0.000084,0.000086,0.000054,0.000032,0.000079


1.5100939273834229

In [14]:
start_time = time.time()

readings_600_nested_list = await reader_cap.backend.read_absorbance(
    wells=plate.children[:],
    wavelength = 600, # units: nm
    output_nested_list=True,
    num_measurement_replicates=5
)
display(pd.DataFrame(readings_600_nested_list))

time.time() - start_time

,0,1,2,3,4,5,6,7,8,9,10,11
0,0.000078,0.000065,0.000078,0.000099,0.000081,0.000080,0.000079,0.000088,0.000055,0.000101,0.000070,0.000081
1,0.000044,0.000061,0.000076,0.000054,0.000054,0.000065,0.000054,0.000069,0.000055,0.000092,0.000046,0.000070
2,0.000072,0.000052,0.000030,0.000073,0.000066,0.000080,0.000055,0.000095,0.000052,0.000056,0.000050,0.000068
3,0.000085,0.000068,0.000052,0.000075,0.000092,0.000057,0.000088,0.000085,0.000071,0.000071,0.000062,0.000059
4,0.000094,0.000062,0.000162,0.000079,0.000080,0.000051,0.000086,0.000106,0.000103,0.000080,0.000060,0.000072
5,0.000041,0.000065,0.000029,0.000068,0.000021,0.000051,0.000028,0.000047,0.000050,0.000095,0.000041,0.000039
6,0.000069,0.000048,0.000020,0.000082,0.000058,0.000057,0.000044,0.000078,0.000050,0.000052,0.000037,0.000062
7,0.000086,0.000057,0.000076,0.000071,0.000066,0.000033,0.000048,0.000086,0.000081,0.000060,0.000048,0.000079


5.985895156860352

In [15]:
first_n_columns = 8

readings_420 = await reader_cap.backend.read_absorbance(
    wells=plate.children[:8*first_n_columns],
    wavelength = 420 # units: nm
)
readings_600 = await reader_cap.backend.read_absorbance(
    wells=plate.children[:8*first_n_columns],
    wavelength = 600 # units: nm
)

well_indexed_df = pd.DataFrame([readings_420, readings_600], index=["420nm", "600nm"]).T
well_indexed_df

,420nm,600nm
A1,0.000064,0.000100
B1,0.000097,0.000033
C1,0.000165,0.000086
D1,0.000105,0.000082
E1,0.000106,0.000132
...,...,...
D8,0.000073,0.000117
E8,0.000085,0.000107
F8,0.000057,0.000053
G8,0.000124,0.000102


## Disconnect from Reader

In [16]:
await reader_cap.stop()